In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

import warnings
warnings.filterwarnings(action = 'ignore')

In [3]:
df = pd.read_csv('combined.csv', index_col='Customer_No')

In [4]:
df1 = df.copy()

In [5]:
df.head()

,N_Default_L3m,Branch_Code,Max_Utilization,Max_Perc_Def_Chg_Pending,N_Family_Member,Max_Ratio_OC_Pending_POS,N_PosBkt_L3m,CoAp_Income,Ever_Default_L12M,Perc_Paymode_Online,...,Birth_Year,Max_DPD_L3m,Perc_Paymode_Cheq_Fail,Age,Total_Field_Trails,Total_Resolved,N_Enq_L9m,Max_Loan_Balance_Others,N_WorkEx_Yr,Target_Flag
Customer_No,,,,,,,,,,,,,,,,,,,,,
1,3,61749,38,100.0,5,NaN,0.0,NaN,Yes,0.285714,...,1977,19.0,0.750,46,NaN,NaN,0,5527.0,15,0.0
2,1,30831,68,100.0,1,NaN,0.0,NaN,Yes,0.000000,...,1990,2.0,0.500,33,NaN,NaN,0,12104.0,2,0.0
3,1,56708,39,50.0,-3,NaN,0.0,NaN,Yes,0.200000,...,1995,25.0,1.000,28,NaN,0.0,1,153179.0,8,0.0
4,0,43551,76,NaN,0,NaN,0.0,NaN,No,0.000000,...,1997,0.0,NaN,26,NaN,NaN,2,181589.0,6,0.0
5,3,15287,23,33.0,11,NaN,0.0,NaN,Yes,0.250000,...,1995,19.0,0.375,28,NaN,NaN,1,10661.0,8,0.0


In [14]:
def missingdata_percentage(df, per=0):
    missing = pd.DataFrame(columns=['category','percentage'])
    for col in df.columns:
        if df[col].isna().values.any():
            count = df[col].isna().sum()
            percentage = 100*df[col].isna().sum()/df.shape[0]
            if percentage>=per:
              missing = missing.append({'category' : col, 'percentage' : percentage, 'count' : int(count)}, ignore_index=True)
    missing.sort_values(by="percentage",ascending=True)
    return missing

In [16]:
application_data_Miss = missingdata_percentage(df)
application_data_Miss

,category,percentage,count
0,Max_Perc_Def_Chg_Pending,25.555635,28447.0
1,Max_Ratio_OC_Pending_POS,89.446071,99566.0
2,N_PosBkt_L3m,6.458307,7189.0
3,CoAp_Income,94.966491,105711.0
4,Max_DPD_L3m,0.324308,361.0
5,Perc_Paymode_Cheq_Fail,28.824766,32086.0
6,Total_Field_Trails,67.412006,75039.0
7,Total_Resolved,61.967048,68978.0
8,Max_Loan_Balance_Others,0.406957,453.0
9,Target_Flag,27.005588,30061.0


category	percentage
0	Max_Perc_Def_Chg_Pending	25.941196
1	Max_Ratio_OC_Pending_POS	89.407160
2	N_PosBkt_L3m	6.569604
3	CoAp_Income	94.931879
4	Max_DPD_L3m	0.299066
5	Perc_Paymode_Cheq_Fail	28.893702
6	Total_Field_Trails	67.335360
7	Total_Resolved	61.911560
8	Max_Loan_Balance_Others	0.419677

category	percentage	count
0	Max_Perc_Def_Chg_Pending	24.513489	7369.0
1	Max_Ratio_OC_Pending_POS	89.551246	26920.0
2	N_PosBkt_L3m	6.157480	1851.0
3	CoAp_Income	95.060045	28576.0
4	Max_DPD_L3m	0.392535	118.0
5	Perc_Paymode_Cheq_Fail	28.638435	8609.0
6	Total_Field_Trails	67.619174	20327.0
7	Total_Resolved	62.117029	18673.0
8	Max_Loan_Balance_Others	0.372576	112.0

In [9]:
df_test = pd.read_csv('Validation Data file.csv', index_col='Customer_No')

In [17]:
test_missing_data = missingdata_percentage(df_test)
test_missing_data

,category,percentage,count
0,Max_Perc_Def_Chg_Pending,24.513489,7369.0
1,Max_Ratio_OC_Pending_POS,89.551246,26920.0
2,N_PosBkt_L3m,6.157480,1851.0
3,CoAp_Income,95.060045,28576.0
4,Max_DPD_L3m,0.392535,118.0
5,Perc_Paymode_Cheq_Fail,28.638435,8609.0
6,Total_Field_Trails,67.619174,20327.0
7,Total_Resolved,62.117029,18673.0
8,Max_Loan_Balance_Others,0.372576,112.0
